In [11]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2

In [7]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [8]:
access_key = config.get('IAM', 'ACCESS_KEY')
secret_key = config.get('IAM', 'SECRET_KEY')
region = config.get('REGION', 'REGION_NAME')

aws_rds_conn = boto3.client('rds', 
                            aws_access_key_id=access_key, 
                            aws_secret_access_key=secret_key, 
                            region_name=region)

In [9]:
db_instance_id = config.get('DB', 'DB_INSTANCE_ID')
db_name = config.get('DB', 'DB_NAME')
db_username = config.get('DB', 'DB_USERNAME')
db_password = config.get('DB', 'DB_PASSWORD')
db_port = config.get('DB', 'DB_PORT')

vpc_security_group = config.get('VPC', 'SECURITY_GROUP')

try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=db_instance_id,
        DBName=db_name,
        MasterUsername=db_username,
        MasterUserPassword=db_password,
        Port=int(db_port),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        PubliclyAccessible=True,
        AllocatedStorage=20,
        VpcSecurityGroupIds=[vpc_security_group]
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print(f"Error: ${ex}")

{'DBInstance': {'DBInstanceIdentifier': 'shop', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'admin_postgress', 'DBName': 'shop', 'AllocatedStorage': 20, 'PreferredBackupWindow': '06:19-06:49', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-475da528', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-a0b0bdc8', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-cb70ba87', 'SubnetAvailabilityZone': {'Name': 'us-east-2c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-436b0939', 'SubnetAvailabilityZone': {'Name': 'us-east-2b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-7e4c7816', 'SubnetAvailabilityZone': {'Na

In [12]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=db_instance_id)
    RDS_HOST_NAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOST_NAME)
except Exception as ex:
    print('Error: ', ex)

shop.cnkf6pwg5si3.us-east-2.rds.amazonaws.com


In [17]:
import create_transactional

try:
    db_pg_conn = psycopg2.connect(
        database=db_name,
        user=db_username,
        password=db_password,
        port=db_port,
        host=RDS_HOST_NAME
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(create_transactional.DDL_QUERY)
    db_pg_conn.commit()
except Exception as ex:
    print('Error: ', ex)

In [18]:
from faker import Faker
import random

fake = Faker()

# Generar datos para "categoria"
def generar_categoria():
    return {
        'nombre': fake.word(),
        'descripcion': fake.text(max_nb_chars=50),
        'estado': fake.boolean()
    }

# Generar datos para "articulo"
def generar_articulo(idcategoria):
    return {
        'idcategoria': idcategoria,
        'codigo': fake.bothify(text='???-###'),
        'nombre': fake.word(),
        'precio_venta': round(random.uniform(10, 1000), 2),
        'stock': fake.random_int(min=0, max=100),
        'descripcion': fake.text(max_nb_chars=50),
        'imagen': fake.file_name(category='image'),
        'estado': fake.boolean()
    }

# Generar datos para "persona"
def generar_persona():
    tipo_persona = random.choice(['cliente', 'proveedor'])
    tipo_documento = random.choice(['DNI', 'CE', 'PAS'])
    return {
        'tipo_persona': tipo_persona,
        'nombre': fake.name(),
        'tipo_documento': tipo_documento,
        'num_documento': fake.bothify(text='########'),
        'direccion': fake.address(),
        'telefono': fake.phone_number(),
        'email': fake.email()
    }

# Generar datos para "rol"
def generar_rol():
    return {
        'nombre': fake.word(),
        'descripcion': fake.text(max_nb_chars=50),
        'estado': fake.boolean()
    }

# Generar datos para "usuario"
def generar_usuario(idrol):
    tipo_documento = random.choice(['DNI', 'CE', 'PAS'])
    return {
        'idrol': idrol,
        'nombre': fake.name(),
        'tipo_documento': tipo_documento,
        'num_documento': fake.bothify(text='########'),
        'direccion': fake.address(),
        'telefono': fake.phone_number(),
        'email': fake.email(),
        'clave': fake.binary(length=16),
        'estado': fake.boolean()
    }

# Generar datos para "venta"
def generar_venta(idcliente, idusuario):
    tipo_comprobante = random.choice(['Factura', 'Boleta'])
    return {
        'idcliente': idcliente,
        'idusuario': idusuario,
        'tipo_comprobante': tipo_comprobante,
        'serie_comprobante': fake.bothify(text='???'),
        'num_comprobante': fake.bothify(text='#######'),
        'fecha': fake.date_time_this_year(),
        'impuesto': round(random.uniform(0, 30), 2),
        'total': round(random.uniform(100, 1000), 2),
        'estado': random.choice(['Pagado', 'Pendiente', 'Anulado'])
    }

# Generar datos para "detalle_venta"
def generar_detalle_venta(idventa, idarticulo):
    return {
        'idventa': idventa,
        'idarticulo': idarticulo,
        'cantidad': fake.random_int(min=1, max=10),
        'precio': round(random.uniform(10, 100), 2),
        'descuento': round(random.uniform(0, 10), 2)
    }

# Generar datos para "ingreso"
def generar_ingreso(idproveedor, idusuario):
    tipo_comprobante = random.choice(['Factura', 'Boleta', 'Ticket'])
    return {
        'idproveedor': idproveedor,
        'idusuario': idusuario,
        'tipo_comprobante': tipo_comprobante,
        'serie_comprobante': fake.bothify(text='???'),
        'num_comprobante': fake.bothify(text='#######'),
        'fecha': fake.date_time_this_year(),
        'impuesto': round(random.uniform(0, 30), 2),
        'total': round(random.uniform(100, 1000), 2),
        'estado': random.choice(['Recibido', 'Pendiente', 'Anulado'])
    }

# Generar datos para "detalle_ingreso"
def generar_detalle_ingreso(idingreso, idarticulo):
    return {
        'idingreso': idingreso,
        'idarticulo': idarticulo,
        'cantidad': fake.random_int(min=1, max=100),
        'precio': round(random.uniform(10, 100), 2)
    }


